<a href="https://colab.research.google.com/github/TroTusk/dino_schisto/blob/main/dino_schisto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#!wget "https://zenodo.org/records/10908163/files/DinoBloom-S.pth?download=1" -O dinobloom-s.pth
#!wget "https://zenodo.org/records/10908163/files/DinoBloom-L.pth?download=1" -O dinobloom-l.pth
!wget "https://zenodo.org/records/10908163/files/DinoBloom-B.pth?download=1" -O dinobloom-b.pth
!git clone https://github.com/zxaoyou/segmentation_WBC.git

--2024-09-15 09:53:55--  https://zenodo.org/records/10908163/files/DinoBloom-B.pth?download=1
Resolving zenodo.org (zenodo.org)... 188.184.103.159, 188.185.79.172, 188.184.98.238, ...
Connecting to zenodo.org (zenodo.org)|188.184.103.159|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 528094030 (504M) [application/octet-stream]
Saving to: ‘dinobloom-b.pth’

dinobloom-b.pth     100%[===================>] 503.63M  22.6MB/s    in 25s     

2024-09-15 09:54:21 (20.3 MB/s) - ‘dinobloom-b.pth’ saved [528094030/528094030]

Cloning into 'segmentation_WBC'...
remote: Enumerating objects: 898, done.
remote: Total 898 (delta 0), reused 0 (delta 0), pack-reused 898 (from 1)
Receiving objects: 100% (898/898), 26.08 MiB | 32.25 MiB/s, done.
Resolving deltas: 100% (112/112), done.


In [ ]:


import numpy as np
from pathlib import Path
# PCA for feature inferred
from sklearn.decomposition import PCA
from skimage.filters import threshold_otsu
import os
import random
from glob import glob

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import matplotlib.pyplot as plt
from PIL import Image
#import rearrange from einops
from einops import rearrange

embed_sizes={"dinov2_vits14": 384,
        "dinov2_vitb14": 768,
        "dinov2_vitl14": 1024,
        "dinov2_vitg14": 1536}

IMAGENET_DEFAULT_MEAN = (0.485, 0.456, 0.406)
IMAGENET_DEFAULT_STD = (0.229, 0.224, 0.225)

patch_num_h=16
patch_num_w=16
img_size=224
eval_model="dinov2_vitb14"

def get_dino_bloom(modelpath="/content/dinobloom-b.pth",modelname="dinov2_vitb14"):
    # load the original DINOv2 model with the correct architecture and parameters.
    model=torch.hub.load('facebookresearch/dinov2', modelname)
    # load finetuned weights
    pretrained = torch.load(modelpath, map_location=torch.device('cpu'))
    # make correct state dict for loading
    new_state_dict = {}
    for key, value in pretrained['teacher'].items():
        if 'dino_head' in key or "ibot_head" in key:
            pass
        else:
            new_key = key.replace('backbone.', '')
            new_state_dict[new_key] = value

    #corresponds to 224x224 image. patch size=14x14 => 16*16 patches
    pos_embed = nn.Parameter(torch.zeros(1, 257, embed_sizes["dinov2_vitb14"]))
    model.pos_embed = pos_embed

    model.load_state_dict(new_state_dict, strict=True)
    return model

dinov2_model=get_dino_bloom()

Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/swiglu_ffn.py:51: UserWarning: xFormers is not available (SwiGLU)
  warnings.warn("xFormers is not available (SwiGLU)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/attention.py:33: UserWarning: xFormers is not available (Attention)
  warnings.warn("xFormers is not available (Attention)")
/root/.cache/torch/hub/facebookresearch_dinov2_main/dinov2/layers/block.py:40: UserWarning: xFormers is not available (Block)
  warnings.warn("xFormers is not available (Block)")
Downloading: "https://dl.fbaipublicfiles.com/dinov2/dinov2_vitb14/dinov2_vitb14_pretrain.pth" to /root/.cache/torch/hub/checkpoints/dinov2_vitb14_pretrain.pth
100%|██████████| 330M/330M [00:01<00:00, 254MB/s]
<ipython-input-3-b52e27bc68dc>:27: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value

In [ ]:



#[TASK 2] Provare con diverse configurazioni di parametri
'''
Modelli disponibili
# DINOv2
dinov2_vits14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')
dinov2_vitb14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14')
dinov2_vitl14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14')
dinov2_vitg14 = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14')

# DINOv2 with registers
dinov2_vits14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14_reg')
dinov2_vitb14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitb14_reg')
dinov2_vitl14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitl14_reg')
dinov2_vitg14_reg = torch.hub.load('facebookresearch/dinov2', 'dinov2_vitg14_reg')
'''
# Step 1: Load the DINOv2 model and modify for SIZExSIZE input
#dinov2_model = torch.hub.load('facebookresearch/dinov2', 'dinov2_vits14')



dinov2_model.eval()  # Set the model to evaluation mode

FREEZE_DINO = True
SIZE = 448 # 224, 448, 672
MODEL_SIZE = 768 #cambiare se cambia il modello s 384  b 768 l 1024 g 1536 eccetera

#HINT se da errore out of memory ridurre questo parametro
BATCH_SIZE = 16

NUM_EPOCHS = 30

# Step 2: Freeze DINOv2 model parameters
if FREEZE_DINO:
  for param in dinov2_model.parameters():
      param.requires_grad = False
x = torch.rand(2, 3, SIZE,SIZE)

#print shape of the features
print(dinov2_model.forward_features(x)['x_norm_patchtokens'].shape)

torch.Size([2, 1024, 768])


In [ ]:
#segmentation head model
class SegmentationHead( nn.Module ):
    def __init__( self, in_channels, out_channels,dino ):
        super( SegmentationHead, self ).__init__()
        self.extrator = dino
        self.to_label = nn.Sequential(
            nn.Linear( in_channels, in_channels ),
            nn.Linear( in_channels, out_channels ),
        )
        self.cnn_deconder_layer_num = 4
        self.inner_channels = 16
        self.drop = 0.0
        self.cnn_deconder_layer = nn.Sequential(
            #nn.BatchNorm2d(self.inner_channels),
            nn.Dropout(self.drop),
            nn.Conv2d(self.inner_channels, self.inner_channels, kernel_size=3, stride=1, padding=1),
            nn.GELU()
            )
        self.cnn_deconder_layers = nn.ModuleList( [self.cnn_deconder_layer] * self.cnn_deconder_layer_num )
        self.exp_conv = nn.Conv2d( 1, self.inner_channels, kernel_size=7, stride=1, padding=3 )
        self.dec_conv = nn.Conv2d( self.inner_channels, 1, kernel_size=7, stride=1, padding=3 )

    def forward( self, x ):
        x = self.extrator.forward_features(x)['x_norm_patchtokens']
        x = self.to_label( x )
        #rearrange
        # Assuming that the ViT divides the image into non-overlapping patches of size (patch_size x patch_size)
        patch_size = 14  # Replace with the actual patch size used in the ViT

        # Calculate the number of patches along each dimension
        n_patches = SIZE // patch_size  # Assuming the input image is SIZExSIZE

        # Reshape and rearrange the patches to reconstruct the image
        x = x.view(x.shape[0], n_patches, n_patches, patch_size, patch_size)
        x = x.permute(0, 1, 3, 2, 4)  # Permute dimensions to correctly align patches
        x = x.contiguous().view(x.shape[0], SIZE, SIZE)  # Reshape to the original image size
        #deconding
        x = x.unsqueeze(1)
        x = self.exp_conv(x)
        for layer in self.cnn_deconder_layers:
            x = layer(x)
        x = self.dec_conv(x)
        x = torch.sigmoid(x)

        return x


seg = SegmentationHead( MODEL_SIZE, 196, dinov2_model ).cuda()
#print model summary
print(seg)

#print shape of the features
#print(seg(x).shape)
class DiceLoss(nn.Module):
    def __init__(self, smooth=1.0):
        super(DiceLoss, self).__init__()
        self.smooth = smooth

    def forward(self, preds, targets):
        # Flatten the tensors
        preds = preds.contiguous().view(-1)
        targets = targets.contiguous().view(-1)

        # Calculate intersection
        intersection = (preds * targets).sum()

        # Calculate Dice coefficient
        dice_coeff = (2. * intersection + self.smooth) / (preds.sum() + targets.sum() + self.smooth)

        # Calculate Dice loss
        dice_loss = 1 - dice_coeff

        return dice_loss
loss = DiceLoss()
#y = torch.rand(2, SIZE,SIZE)
#print loss
#print(loss(seg(x), y))
#backward
#loss(seg(x), y).backward()

#print number of trainable parameters and total number
total_params = sum(p.numel() for p in seg.parameters())
trainable_params = sum(p.numel() for p in seg.parameters() if p.requires_grad)

print(f"Total parameters: {total_params}")
print(f"Trainable parameters: {trainable_params}")




SegmentationHead(
  (extrator): DinoVisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(14, 14), stride=(14, 14))
      (norm): Identity()
    )
    (blocks): ModuleList(
      (0-11): 12 x NestedTensorBlock(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): MemEffAttention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): LayerScale()
        (drop_path1): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (drop): Dropout(p=0.0, inplace=False)
     

In [ ]:
#load the image asd.jpg and resize to SIZExSIZE
#[TASK 3] Provare diverse tecniche di augmentation vedi  https://albumentations.ai/docs/getting_started/mask_augmentation/
#Nota bene che per "augmentare" le immagini del dataset devi lavorare simultaneamente su maschera di segmentazione e immagine per
#evitare errori trovi sotto le librerie commentate per iniziare a usare tale libreria
#import albumentations as A
#from albumentations.pytorch import ToTensorV2
def process(img):
    img = Image.open(img).convert('RGB')
    # Define the transformations
    transform_pipeline = transforms.Compose([
        transforms.Resize((SIZE, SIZE)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    # Apply transformations
    img = transform_pipeline(img)



    return img



In [ ]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#import dataset and dataloader from torch
from torch.utils.data import Dataset, DataLoader,random_split
import os

#create the dataset that output both image and mask

class Custom_Dataset(Dataset):
    def __init__(self, img_dir, mask_dir, preproceesor):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.preproceesor = preproceesor

        self.images = os.listdir(img_dir)


    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_name = self.images[idx]
        img_path = os.path.join(self.img_dir, img_name)
        mask_path = os.path.join(self.mask_dir, img_name.replace('.jpg', '_mask.png'))


        mask = Image.open(mask_path).convert('L') # Assuming masks are single-channel grayscale

        mask = transforms.Resize((SIZE, SIZE))(mask)
        mask = transforms.ToTensor()(mask)
        image = self.preproceesor(img_path)

        return image, mask
#split into train val test
# TASK [0] adattare il dataset Schistociti al formato richiesto, vedi esempio dataset persone
# TASK [1] adattare al dataset Schistociti
img_dir = '/content/drive/MyDrive/Colab Notebooks/dataset_schisto_coco/train/images'
mask_dir = '/content/drive/MyDrive/Colab Notebooks/dataset_schisto_coco/train/masks'


dataset = Custom_Dataset(img_dir=img_dir, mask_dir=mask_dir, preproceesor=process)


In [ ]:
train_size = int(0.7 * len(dataset))
val_size = int(0.15 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader for each set

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
model=seg.cuda()
from torch.optim import AdamW

optimizer = AdamW(model.parameters(), lr=0.001)


In [ ]:
import numpy as np
#leggere https://medium.com/@nghihuynh_37300/understanding-evaluation-metrics-in-medical-image-segmentation-d289a373a3f
def precision_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_pred = np.sum(pred_mask)
    precision = np.mean(intersect/total_pixel_pred)
    return round(precision, 3)

def recall_score_(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_pixel_truth = np.sum(groundtruth_mask)
    recall = np.mean(intersect/total_pixel_truth)
    return round(recall, 3)

def accuracy(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    union = np.sum(pred_mask) + np.sum(groundtruth_mask) - intersect
    xor = np.sum(groundtruth_mask==pred_mask)
    acc = np.mean(xor/(union + xor - intersect))
    return round(acc, 3)

def dice_coef(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    total_sum = np.sum(pred_mask) + np.sum(groundtruth_mask)
    dice = np.mean(2*intersect/total_sum)
    return round(dice, 3) #round up to 3 decimal places

def iou(groundtruth_mask, pred_mask):
    intersect = np.sum(pred_mask*groundtruth_mask)
    union = np.sum(pred_mask) + np.sum(groundtruth_mask) - intersect
    iou = np.mean(intersect/union)
    return round(iou, 3)

def compute_metrics(groundtruth_mask, pred_mask):
    pre = precision_score_(groundtruth_mask, pred_mask)
    rec = recall_score_(groundtruth_mask, pred_mask)
    acc = accuracy(groundtruth_mask, pred_mask)
    dice = dice_coef(groundtruth_mask, pred_mask)
    iou_ = iou(groundtruth_mask, pred_mask)

    #print(f"Precision: {pre} , Recall: {rec} , Accuracy: {acc} , Dice: {dice} , IoU: {iou_}")
    return pre, rec, acc, dice, iou_


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#DiceLoss()
#nn.L1Loss()
#nn.BCELoss()
#nn.BCEWithLogitsLoss()
#criterion = nn.MSELoss()
criterion = DiceLoss()
model.to(device)
best_val_loss = float('inf')


for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0.0

    for images, masks in train_loader:
        images, masks = images.to(device), masks.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, masks)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Loss: {running_loss/len(train_loader):.4f}')

    # Validation phase
    model.eval()
    a_pre = []
    a_rec = []
    a_acc = []
    a_dice = []
    a_iou = []
    val_loss = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)
            outputs = model(images)
            loss = criterion(outputs, masks)
            val_loss += loss.item()
            #compute metrics
            pre, rec, acc, dice, iou_ = compute_metrics(masks.cpu().numpy(), outputs.cpu().numpy())
            a_pre.append(pre)
            a_rec.append(rec)
            a_acc.append(acc)
            a_dice.append(dice)
            a_iou.append(iou_)
    print(f"Precision: {np.mean(a_pre)} , Recall: {np.mean(a_rec)} , Accuracy: {np.mean(a_acc)} , Dice: {np.mean(a_dice)} , IoU: {np.mean(a_iou)}")

    # Save the model if validation loss improves
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')

    print(f'Validation Loss after Epoch {epoch+1}: {val_loss/len(val_loader):.4f}')


In [ ]:
model.load_state_dict(torch.load('best_model.pth'))
model.eval()
test_loss = 0.0
a_pre = []
a_rec = []
a_acc = []
a_dice = []
a_iou = []
with torch.no_grad():
    for images, masks in test_loader:
        images, masks = images.to(device), masks.to(device)
        outputs = model(images)
        loss = criterion(outputs, masks)
        test_loss += loss.item()
        pre, rec, acc, dice, iou_ = compute_metrics(masks.cpu().numpy(), outputs.cpu().numpy())
        a_pre.append(pre)
        a_rec.append(rec)
        a_acc.append(acc)
        a_dice.append(dice)
        a_iou.append(iou_)
print(f"Precision: {np.mean(a_pre)} , Recall: {np.mean(a_rec)} , Accuracy: {np.mean(a_acc)} , Dice: {np.mean(a_dice)} , IoU: {np.mean(a_iou)}")
print(f'Test Loss: {test_loss/len(test_loader):.4f}')


In [ ]:
#plot last label and prediction in a single plot 2x1

def plot_label_and_prediction(label, prediction):
    # Convert tensors to numpy arrays for plotting
    label = label.squeeze().cpu().numpy()
    #if greater than 0.5 is 1 else 0

    prediction = prediction.squeeze().cpu().detach().numpy()
    prediction[prediction >= 0.5] = 1
    prediction[prediction < 0.5] = 0

    # Create a 2x1 plot
    fig, axes = plt.subplots(2, 1, figsize=(8, 8))

    # Plot the label (Ground Truth)
    axes[0].imshow(label, cmap='gray')
    axes[0].set_title('Ground Truth (Label)')
    axes[0].axis('off')  # Turn off axis

    # Plot the prediction
    axes[1].imshow(prediction, cmap='gray')
    axes[1].set_title('Prediction')
    axes[1].axis('off')  # Turn off axis

    # Adjust layout
    plt.tight_layout()
    plt.show()



for i in range(len(masks)):
  plot_label_and_prediction(masks[i], outputs[i])
